# Pairwise Experiments

### Setup

In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.5 MB/s eta 0:00:00


In [ ]:

import os
os.environ["GROQ_API_KEY"] = ""   
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"


from langsmith import Client
from pydantic import BaseModel, Field
from groq import Groq  


client = Client()
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])


dataset = client.clone_public_dataset(
    "https://smith.langchain.com/public/9078d2f1-7bef-4ba7-b795-210a17682ef9/d"
)

SUMMARIZATION_SYSTEM_PROMPT = """You are a judge, aiming to score how well a summary summarizes the content of a transcript"""

SUMMARIZATION_HUMAN_PROMPT = """
[The Meeting Transcript] {transcript}
[The Start of Summarization] {summary} [The End of Summarization]"""


class SummarizationScore(BaseModel):
    score: int = Field(description="""A score from 1-5 ranking how good the summarization is for the provided transcript, with 1 being a bad summary, and 5 being a great summary""")

def summary_score_evaluator(inputs: dict, outputs: dict) -> dict:
    completion = groq_client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": SUMMARIZATION_SYSTEM_PROMPT},
            {
                "role": "user",
                "content": SUMMARIZATION_HUMAN_PROMPT.format(
                    transcript=inputs["transcript"],
                    summary=outputs.get("output", "N/A")
                )
            }
        ]
    )

    response_text = completion.choices[0].message.content.strip()


    try:
        score = int(response_text[0])  
    except Exception:
        score = 1  

    return {"key": "summary_score", "score": score}



def good_summarizer(inputs: dict):
    response = groq_client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {
                "role": "user",
                "content": f"Concisely summarize this meeting in 3 sentences. Make sure to include all of the important events. Meeting: {inputs['transcript']}"
            }
        ]
    )
    return response.choices[0].message.content

client.evaluate(
    good_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Good Summarizer"
)


def bad_summarizer(inputs: dict):
    response = groq_client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {
                "role": "user",
                "content": f"Summarize this in one sentence. {inputs['transcript']}"
            }
        ]
    )
    return response.choices[0].message.content

client.evaluate(
    bad_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Bad Summarizer"
)


View the evaluation results for experiment: 'Good Summarizer-2ba84924' at:
https://smith.langchain.com/o/fbc0b987-03c7-4ba9-82d3-c46df3d88773/datasets/ef3f2810-af28-4335-b6d6-f63d90b9752b/compare?selectedSessions=f8cf1034-3e55-4465-b2f2-bc00e1c0c080




0it [00:00, ?it/s]

View the evaluation results for experiment: 'Bad Summarizer-3908e77a' at:
https://smith.langchain.com/o/fbc0b987-03c7-4ba9-82d3-c46df3d88773/datasets/ef3f2810-af28-4335-b6d6-f63d90b9752b/compare?selectedSessions=b2fbc8d4-ae64-4c0e-8168-7d1f53b3ce44




0it [00:00, ?it/s]

,inputs.transcript,outputs.output,error,feedback.summary_score,execution_time,example_id,id
0,"Bob and Ms. Thompson (NO DEAL): Bob: Hi, Ms. T...","Bob greets Ms. Thompson at Ford Motors, discus...",None,1,0.639239,0d3faa06-056d-4b85-9112-7d9776ca1a17,e8cb18f6-b3bc-4e84-a840-ca52e7700208
1,Bob and Mr. Johnson (CLOSED DEAL): Bob: Good m...,Bob convinces Mr. Johnson to buy a Ford Explor...,None,1,0.452373,16764f66-cea4-46ed-b820-42af3e60edea,c716e8bd-33f1-4f3a-978c-0466d73ad865
2,Bob and Ms. Nguyen (NO DEAL): Bob: Good aftern...,Bob tried to match Ms. Nguyen’s interest in an...,None,1,0.435999,4c37ae13-3ae4-4839-a95b-8443a9cf3848,aba49f42-4ab2-4a75-9835-add08c7fe779
3,"Bob and Mr. Patel (CLOSED DEAL): Bob: Hello, M...",Bob helped Mr. Patel choose and purchase a For...,None,1,4.645834,4cbddbf1-d38f-4058-8cb7-6f9744027905,1d007c5a-a27c-4f2d-92ea-425175d4fe53
4,Bob and Mr. Carter (CLOSED DEAL): Bob: Welcome...,Bob helps Mr. Carter trade his 2015 Tacoma for...,None,1,4.499393,84283f28-8083-4b77-a5ce-93ccc6b7d14f,771cae8e-1a4b-4249-83ae-a34bf506ae0d


In [7]:
# -------------------------------
# Pairwise Evaluator with Groq
# -------------------------------

JUDGE_SYSTEM_PROMPT = """
Please act as an impartial judge and evaluate the quality of the summarizations provided by two AI summarizers to the meeting transcript below.
Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their summarizations.
Begin your evaluation by comparing the two summarizations and provide a short explanation.
Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision.
Do not favor certain names of the assistants.
Be as objective as possible.
"""

JUDGE_HUMAN_PROMPT = """
[The Meeting Transcript] {transcript}

[The Start of Assistant A's Summarization] {answer_a} [The End of Assistant A's Summarization]

[The Start of Assistant B's Summarization] {answer_b} [The End of Assistant B's Summarization]
"""

def ranked_preference(inputs: dict, outputs: list[dict]) -> list:
    # Create the Groq chat completion
    completion = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {
                "role": "system",
                "content": JUDGE_SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": JUDGE_HUMAN_PROMPT.format(
                    transcript=inputs["transcript"],
                    answer_a=outputs[0].get("output", "N/A"),
                    answer_b=outputs[1].get("output", "N/A")
                )
            }
        ]
    )

    # Extract the raw text
    response_text = completion.choices[0].message.content.strip()

    # Attempt to detect preference manually
    preference_score = 0
    if "1" in response_text.split()[0]:
        preference_score = 1
    elif "2" in response_text.split()[0]:
        preference_score = 2
    else:
        preference_score = 0  # Tie or unclear

    # Convert to a list of scores per assistant
    if preference_score == 1:
        scores = [1, 0]
    elif preference_score == 2:
        scores = [0, 1]
    else:
        scores = [0, 0]

    return scores

# -------------------------------
# Run pairwise evaluation
# -------------------------------
from langsmith import evaluate

evaluate(
    ("Good Summarizer-2ba84924", "Bad Summarizer-3908e77a"),  # Replace with your actual experiment IDs
    evaluators=[ranked_preference]
)


View the pairwise evaluation results at:
https://smith.langchain.com/o/fbc0b987-03c7-4ba9-82d3-c46df3d88773/datasets/ef3f2810-af28-4335-b6d6-f63d90b9752b/compare?selectedSessions=ca050055-e6a5-4f27-87f3-567d13bcccfd%2Ca39da72d-e44d-45c5-b1cf-f8e68439c747&comparativeExperiment=890eb6dc-3b71-4545-bd38-603d5bc8c040




  0%|          | 0/5 [00:00<?, ?it/s]

### Task

Let's set up a new task! Here, we have a salesperson named Bob. Bob has a lot of deals, so he wants to summarize what happened in this deals based off of some meeting transcripts.

Bob is iterating on a few different prompts, that will give him nice, concise transcripts for his deals.

Bob has curated a dataset of his deal transcripts, let's go ahead and load that in. You can take a look at the dataset as well if you're curious! Note that this is not a golden dataset, there is no reference output here.

### Experiments

Now, let's run some experiments on this dataset using two different prompts. Let's add an evaluator that tries to score how good our summaries are!

First, we'll run our experiment with a good version of our prompt!

Now, we'll run an experiment with a worse version of our prompt, to highlight the difference.

### Pairwise Experiment

Let's define a function that will compare our two experiments. These are the fields that pairwise evaluator functions get access to:
- `inputs: dict`: A dictionary of the inputs corresponding to a single example in a dataset.
- `outputs: list[dict]`: A list of the dict outputs produced by each experiment on the given inputs.
- `reference_outputs: dict`: A dictionary of the reference outputs associated with the example, if available.
- `runs: list[Run]`: A list of the full Run objects generated by the experiments on the given example. Use this if you need access to intermediate steps or metadata about each run.
- `example: Example`: The full dataset Example, including the example inputs, outputs (if available), and metdata (if available).

First, let's give our LLM-as-Judge some instructions. In our case, we're just going to directly use LLM-as-judge to grade which of the summarizers is the most helpful.

It might be hard to grade our summarizers without a ground truth reference, but here, comparing different prompts head to head will give us a sense of which is better!

Our function will take in an `inputs` dictionary, and a list of `outputs` dictionaries for the different experiments that we want to compare.

Now let's run our pairwise experiment with `evaluate()`